In [14]:
import os
import json
import pandas as pd
import plotly.graph_objects as go

In [15]:
ROOT = !pwd
data_path = os.path.join(ROOT[0], '../', "data", "almrrc2021-data-training", "model_apply_inputs",)

In [33]:
with open(os.path.join(data_path, "new_route_data.json"), "r") as f:
    data = json.load(f)
with open(os.path.join(data_path, "new_travel_times.json"), "r") as f:
    time_data = json.load(f)

In [34]:
data.keys()

dict_keys(['RouteID_15baae2d-bf07-4967-956a-173d4036613f', 'RouteID_3f166f0e-fd2e-47ab-96a0-6cbc99cc6eef', 'RouteID_5486294a-503f-4346-b8a9-862e988cbe7c', 'RouteID_693060a6-88bb-4324-9e9c-925d5240263c', 'RouteID_7f5d87f0-c39f-434f-bf3f-b159ef321909', 'RouteID_9475872b-287f-4c2c-8e29-887766a4e090', 'RouteID_a8f0009d-e50a-49c9-84d3-f9885ad14a54', 'RouteID_bcc07fea-86d2-41e4-9a58-cfc78956dcc7', 'RouteID_d1a8c3dd-fa67-455c-a68d-af2fd6aa5d91', 'RouteID_e6687a05-2453-4edc-b86c-7558ab6d93f6', 'RouteID_2b8df66d-fcd4-438e-931c-3b84b36a5c6b', 'RouteID_f3261fad-5f97-44f6-ae7f-cf169f5d6452', 'RouteID_fffd257c-3041-4736-be7a-5efea8af1173'])

In [29]:
df = pd.DataFrame.from_records(
    ({"stop_id": k, **v} for k, v in data['RouteID_15baae2d-bf07-4967-956a-173d4036613f']['stops'].items()))



In [30]:
df.head()

,stop_id,lat,lng,type,zone_id
0,AH,42.129085,-88.027485,Dropoff,D-8.3E
1,AK,42.133454,-88.043144,Dropoff,D-8.2H
2,AN,42.129422,-88.037895,Dropoff,D-8.2H
3,AU,42.107882,-88.035445,Dropoff,D-8.1C
4,AV,42.138752,-88.041745,Dropoff,D-7.1J


In [37]:
travel_times = pd.DataFrame.from_records(
    ({"stop_id": k, "time": v} for k, v in time_data['RouteID_15baae2d-bf07-4967-956a-173d4036613f']['AH'].items()))


In [40]:
new_df = df.merge(travel_times, on='stop_id')
new_df.head()

,stop_id,lat,lng,type,zone_id,time
0,AH,42.129085,-88.027485,Dropoff,D-8.3E,0.0
1,AK,42.133454,-88.043144,Dropoff,D-8.2H,287.4
2,AN,42.129422,-88.037895,Dropoff,D-8.2H,249.4
3,AU,42.107882,-88.035445,Dropoff,D-8.1C,476.9
4,AV,42.138752,-88.041745,Dropoff,D-7.1J,421.2


In [59]:
d = {z: i for i, z in enumerate(df['zone_id'].unique())}
new_df['zone_int'] = new_df['zone_id'].map(d)

In [60]:
new_df['cum_time'] = new_df['time'].cumsum()
new_df['cum_time'] /= 60

In [65]:
down_sampled_df = new_df.groupby('zone_id').agg({'cum_time': 'first', 'lat': 'mean', 'lng': 'mean', 'zone_int': 'first'}).reset_index()

In [ ]:
# create a zone bounding box
for group in new_df.groupby('zone_id'):
    zone_id = group[0]
    df = group[1]
    lat_min = df['lat'].min()
    lat_max = df['lat'].max()
    lng_min = df['lng'].min()
    lng_max = df['lng'].max()
    print(zone_id, lat_min, lat_max, lng_min, lng_max)
    # create a bounding box for the zone
    bbox = go.layout.Shape(
        type="rect",
        x0=lng_min,
        y0=lat_min,
        x1=lng_max,
        y1=lat_max,
        line=dict(
            color="black",
            width=1
        )
    )
    # add the bounding box to the map
    fig.add_shape(bbox)

In [79]:
mapbox_access_token = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"

fig = go.Figure(
    data=go.Scattermapbox(
        lat=down_sampled_df["lat"],
        lon=down_sampled_df["lng"],
        mode="lines+markers",
        marker=go.scattermapbox.Marker(
            # size=new_df['cum_time'] * 10,
            color=new_df["zone_int"],
            # colorbar=go.ColorBar(
            #     title='Time (minutes)'
            # ),
        ),
        text=new_df["cum_time"],
    )
)

# create a zone bounding box
for zone_id, df in new_df.groupby("zone_id"):
    fig.add_trace(
        go.Scattermapbox(
            fill="toself",
            lon=[df["lng"].min(), df["lng"].max(), df["lng"].max(), df["lng"].min()] ,
            lat=[df["lat"].max(), ] *2 + [df["lat"].min(), ] * 2,
            marker={"size": 10, "color": df["zone_int"]},
            text=zone_id,
            showlegend=False
        )
    )

    # add the bounding box to the map
    # fig.add_shape(bbox)

fig.update_layout(
    # autosize=True,
    hovermode="closest",
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        style="mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=new_df["lat"].mean(), lon=new_df["lng"].mean()
        ),
        pitch=0,
        zoom=12,
    ),
)
fig.show()
